In [20]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [21]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [22]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [23]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [24]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [25]:
msg = 'secret message'.encode('utf-8')
msg

b'secret message'

# Key generation

In [26]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [27]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((17163117661735096795825544959140860990484184819747499872751542734731962848009861513981359655264675664973250406205789278507875727258673165305625756076971930752235439560538470113009427557548495557153999715174013374285330533525717036938661166203969056550780667700571940122296913690174032748759729655684595443947346465414671869909851509521621528870947769329228069068636586854142790097712861406401580961507360900056931844014684678629392751647704270088566165508484427237871684268331749749063773916567180318088273523906352236190017598756587302126929152104907724004369251052186097084056454212457059414305162307058155021479967, 9745862409826539788919935066537239796280942550574349403328519914445406877004105341928264824545650335217124249310753644175029564856664670617525413314850676880292291467126916076012282819284655352279732461792202050612991075597831238871264324589919378478091642402433372406836927309833288338085191001654481068743684195592862100147977542363465888898370386985159433792494042051371685640

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [28]:
msg = 'secret message'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

2340509926146499719798124187838309

In [29]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [30]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

17095226430846445495387267174306250956262054574823331523317215999560141626485437938143119253771023600079158488606331687127372551925956352965919418314861251869964185096214290039494607958184359351304683482504619430509411093056958536251572863456675697393263011787379335211831905393467804460885025375962953074359931268653969295196308021146820391001621753859999886649248798943377962071609576985659207961786102869969096007331821668998725092549837481450513626263751127328782754570991968966365506392913425742995101201645473321358571249478557599666849111660159922509975291390570200325884557982138977509270799003883038341238603

In [31]:
y_hack = pow(x_hack, e, n)
y_hack

7442157370935249150826186022125639232719006421071047041933341019901689318346335525258947812502178931344579611707835721422143615663884265834990485495038537877754220379659575429030846138823151444249387887081136197895148084389883545150559905386541545642756011967200076198512016752831933096840329527738641483971317410579098242862649915559386327274696195626977904877248337739553852072511554266353190281242443775082980780379684333871194304994917175094254877195116294050666371531440802164297456553790435370776949223558155982627022934781897947750886499968713222454855512813571664146436962732185098602302540691197638995526600

# Hash

In [32]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')
h

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Signing

In [33]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

490551860375347504812594246456075954140730050836887102974309871542976641480393445282935278027993821937591425291520428751252690021741795888251871067790500362418715905585501189359141261565653104524072711443979720327196679882978348232070314474370826895890814327074398793804460754595868408800626525628073297996108359295731417405068811137757685262429086041869943320161054801517354215955506645706731120099203358391979073257670056557724447021136038549871836020459058467426231881100676952373227518381994425747599670225809605416165523813128822277095711696255569182328885376578139722456889415324886565768543774795326694051282

# Decryption of hash (sign) with Sender pub key

In [34]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Decryption of message

In [35]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg, type(decrypt_to_msg)

(2340509926146499719798124187838309, int)

# Hash verification 

In [36]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(int.from_bytes(final_h.digest()) == decrypt_to_hash)
#print(final_h.digest() == decrypt_to_hash)

True


# Get plaintext

In [37]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret message'

### Testing bitwise operator :((

In [38]:
example_bits = 0b11111111
K = list(range(0,example_bits.bit_length()))
n = example_bits.bit_length()
for k in K:
    print(bin(example_bits), k, bin(example_bits ^ (1 << k)))
n

0b11111111 0 0b11111110
0b11111111 1 0b11111101
0b11111111 2 0b11111011
0b11111111 3 0b11110111
0b11111111 4 0b11101111
0b11111111 5 0b11011111
0b11111111 6 0b10111111
0b11111111 7 0b1111111


8

In [39]:
bin(0b00000001 << 3) # -> 0b00001000

'0b1000'

# 3DES

In [40]:
from Crypto.Cipher import DES3, AES
from Crypto.Random import get_random_bytes

def generate_3des_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
        except ValueError:
            continue
        
        k1, k2, k3 = key[:8], key[8:16], key[16:]
        
        if k1 != k2 and k2 != k3 and k1 != k3:
            return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_3des_key()

print("3DES key (hex):", key.hex())
print("K1:", k1.hex())
print("K2:", k2.hex())
print("K3:", k3.hex())


3DES key (hex): cdd06854efec29e3d3b008fb7a0457a28f85522cb664cb51
K1: cdd06854efec29e3
K2: d3b008fb7a0457a2
K3: 8f85522cb664cb51


In [41]:
from base64 import b64encode, b64decode
from Crypto.Util.Padding import pad, unpad

def encypt_3DES(plaintext, key):
    cipher = DES3.new(key, DES3.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext, DES3.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct

In [42]:
data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')
iv, ct = encypt_3DES(plaintext=data, key=key)

In [43]:
def decrypt_3DES(iv, ct, key):
    try:
        cipher = DES3.new(key, DES3.MODE_CBC, iv= b64decode(iv))
        #pt = unpad(cipher.decrypt(ct), DES3.block_size)
        pt = unpad(cipher.decrypt(b64decode(ct)), DES3.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

In [44]:
decrypt_3DES(iv, ct, key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'


# AES encryption

In [45]:
from Crypto.Cipher import AES
from Crypto.Util import Counter

nonce = get_random_bytes(8)    # 8-byte nonce (64 bits)
key = get_random_bytes(32)     # AES-256 key

ctr_encrypt = Counter.new(64, prefix=nonce)   # 64-bit counter, prefixed with nonce
aes_cipher_encrypt = AES.new(key, AES.MODE_CTR, counter=ctr_encrypt)
ciphertext = aes_cipher_encrypt.encrypt(data)

encoded_ciphertext = b64encode(ciphertext).decode("utf-8")
print("Ciphertext (Base64):", encoded_ciphertext)



Ciphertext (Base64): DCMt2jAjIVRfPmxQqyWSla5amDqvseWKoYDAYQnwAbK9Nw+Qm0iW1Ofduz6S5+gvSN1mexr6Pm/W7jgEKVVUISCAGvAzH57cXK8yP5XWwcyCGUVTJ3Z7sd+OvRF5sa5M+noFbyf2gkC5sbujclGLfDYrrrNfaPg6kJ1lNES+615YMtUVvt6cNcMCoa2OTWTjgxxdltxljxJC1FvTTwEzlwOJnspvrYsZX74WOGpAfO4pSiM760Wjp7FAzCW+kirrFYMz4dOjwNkd+3PS9dJ8nMyPcyzxd4aE2VhJM5/s99aVJXf6dc+P5RqNsBEXNaYFqrJ8MEiSLQnQ5h/+wGH0wriHQPNelj58XvbezRkf0055xgVWsF39NSC0eXk9TW8N6IEXzRZz1ZEPldTU8ZytcN0DBJcG/GtQEuXUs5lgHwvQOt4ja0K6c58EbBIk9ixqEmxUYLHXjEWfKncJiyvQlsh0yjyqyhUVwjq4pRCZ1uuqv8m5FhABzhbB6tlP4qxhmSHSbAlM6erP63XgOZ65TDx6Sthk4RLUbt+kwNIfHOXA3BO8AGMbEeNAgzk2vkOf+i4l/SVs9Zk+hLZl


## AES Decryption

In [46]:

ctr_dec = Counter.new(64, prefix=nonce)
aes_decipher = AES.new(key, AES.MODE_CTR, counter=ctr_dec)
decrypted_bytes = aes_decipher.decrypt(b64decode(encoded_ciphertext))

decrypted_text = decrypted_bytes.decode('utf-8')
print("Decrypted (readable):", decrypted_text)


Decrypted (readable): Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


## Lesson 7 Python Qs

1. Why must you follow the algorithm steps exactly as prescribed? What happens if you skip some step or apply it in different order?
- ????
2. How did you securely obtain the keys for encryption? How would you exchange these keys to the receiver of you message? How long should the key be considered valid (expiration)? How can you estimate/decide when a key is no longer secure enough?
- For DES, we check if k1 not equal to k2 and k3 
- For AES, it is just a random 256 bit key
- We would exchange using public key encryption like RSA
- 1 message session??? like every reply?? verify with source
- Compute moore's law yaaa research lg

3. Why do we use tripple DES and not a double DES? Is double DES stronger than single DES? Would quadrouple DES be safer to use than tripple DES? Explain.
- if double DES, the key length is only 80 bits effective. yes? because key is longer but idk about computer power HRS SEARCH

4. Why is it advised to have a plain text longer than block size? You encrypted a lorem impsum, which is a clear text with a pattern. Why does your encrypted cipher text has no visible patterns in it? Compare what extra steps in AES are taken to ensure that the plain text pattern is concealed (vs. simple substitution cipher).

BACA LG ANJGGG

5. Can encryption of two different texts result in the same cipher text? How this can be done and what implications does it have on security?

NO?? bcs of iv?? or because of ctr block???? GATAUU WOI


